## Data exploration and basic cleaning

In [1]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

In [2]:
df_matches = pd.read_csv("IPL_Matches.csv")

In [3]:
# check the no of data points we have
df_matches.shape

(816, 17)

In [4]:
# let's look at some rows

df_matches.head()

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,N,NaN,MR Benson,SL Shastri
2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,N,NaN,SJ Davis,DJ Harper
4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,N,NaN,BF Bowden,K Hariharan


In [5]:
# column names

df_matches.columns

Index(['id', 'city', 'date', 'player_of_match', 'venue', 'neutral_venue',
       'team1', 'team2', 'toss_winner', 'toss_decision', 'winner', 'result',
       'result_margin', 'eliminator', 'method', 'umpire1', 'umpire2'],
      dtype='object')

In [6]:
# remove some columns that we don't need
# we can also do use pandas drop

df = df_matches[["id","city","date","player_of_match","venue","team1",\
                 "team2","toss_winner","toss_decision","winner","result",\
                 "result_margin","umpire1","umpire2"]]

In [7]:
df.head(2)

,id,city,date,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,umpire1,umpire2
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,Asad Rauf,RE Koertzen
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,MR Benson,SL Shastri


In [8]:
# let's do some white space cleanup first

for col in df.columns.tolist():
    if col != "result_margin":
        df[col] = df[col].apply(lambda x:str(x).strip())

### We have some data quality issues

1. some teams have changed their name

2. some team names are repeating i.e Rising Pune Supergiant and Rising Pune Supergiants

In [9]:
# let's look at all the unique team names we get

set(df["team1"].tolist() + df["team2"].tolist())

{'Chennai Super Kings',
 'Deccan Chargers',
 'Delhi Capitals',
 'Delhi Daredevils',
 'Gujarat Lions',
 'Kings XI Punjab',
 'Kochi Tuskers Kerala',
 'Kolkata Knight Riders',
 'Mumbai Indians',
 'Pune Warriors',
 'Rajasthan Royals',
 'Rising Pune Supergiant',
 'Rising Pune Supergiants',
 'Royal Challengers Bangalore',
 'Sunrisers Hyderabad'}

In [10]:
# we will replace the team names with their latest names
# using a simple dict to do the job

latest_names = {
    "Pune Warriors":"Rising Pune Supergiant",
    "Rising Pune Supergiants":"Rising Pune Supergiant",
    "Deccan Chargers":"Sunrisers Hyderabad",
    "Delhi Daredevils":"Delhi Capitals"
}

In [11]:
# use the above dict to fix names

for col in ["team1","team2","toss_winner","winner"]:
    df[col] = df[col].apply(lambda x:latest_names[x] if x in latest_names else x)

In [12]:
set(df["team1"].tolist() + df["team2"].tolist())

{'Chennai Super Kings',
 'Delhi Capitals',
 'Gujarat Lions',
 'Kings XI Punjab',
 'Kochi Tuskers Kerala',
 'Kolkata Knight Riders',
 'Mumbai Indians',
 'Rajasthan Royals',
 'Rising Pune Supergiant',
 'Royal Challengers Bangalore',
 'Sunrisers Hyderabad'}

In [13]:
df.head(2)

,id,city,date,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,umpire1,umpire2
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,Asad Rauf,RE Koertzen
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,MR Benson,SL Shastri


### Removing NaN values

In [14]:
# check which columns have NaN values and their counts

for col in df.columns:
    print(f"{col} : {df[col].isna().sum()}")

id : 0
city : 0
date : 0
player_of_match : 0
venue : 0
team1 : 0
team2 : 0
toss_winner : 0
toss_decision : 0
winner : 0
result : 0
result_margin : 17
umpire1 : 0
umpire2 : 0


In [15]:
# drop NaN values

df.dropna(inplace=True)

In [16]:
# no of data points left after droping

df.shape

(799, 14)

In [17]:
# recheck

for col in df.columns:
    print(f"{col} : {df[col].isna().sum()}")

id : 0
city : 0
date : 0
player_of_match : 0
venue : 0
team1 : 0
team2 : 0
toss_winner : 0
toss_decision : 0
winner : 0
result : 0
result_margin : 0
umpire1 : 0
umpire2 : 0


In [18]:
# cleaning city names

df['city'].unique()

array(['Bangalore', 'Chandigarh', 'Delhi', 'Mumbai', 'Kolkata', 'Jaipur',
       'Hyderabad', 'Chennai', 'Cape Town', 'Port Elizabeth', 'Durban',
       'Centurion', 'East London', 'Johannesburg', 'Kimberley',
       'Bloemfontein', 'Ahmedabad', 'Cuttack', 'Nagpur', 'Dharamsala',
       'Kochi', 'Indore', 'Visakhapatnam', 'Pune', 'Raipur', 'Ranchi',
       'Abu Dhabi', 'nan', 'Rajkot', 'Kanpur', 'Bengaluru', 'Sharjah',
       'Dubai'], dtype=object)

### Handle the Bangalore and Bengaluru case

**We will change the city name to its latest i.e** `Bengaluru` 

In [19]:
df["city"] = df["city"].apply(lambda x:"Bengaluru" if x == "Bangalore" else x)

In [20]:
df["city"].unique()

array(['Bengaluru', 'Chandigarh', 'Delhi', 'Mumbai', 'Kolkata', 'Jaipur',
       'Hyderabad', 'Chennai', 'Cape Town', 'Port Elizabeth', 'Durban',
       'Centurion', 'East London', 'Johannesburg', 'Kimberley',
       'Bloemfontein', 'Ahmedabad', 'Cuttack', 'Nagpur', 'Dharamsala',
       'Kochi', 'Indore', 'Visakhapatnam', 'Pune', 'Raipur', 'Ranchi',
       'Abu Dhabi', 'nan', 'Rajkot', 'Kanpur', 'Sharjah', 'Dubai'],
      dtype=object)

### Fix some venue names

see [stadiums info](https://en.wikipedia.org/wiki/List_of_international_cricket_grounds_in_India) for more details

In [21]:
for v in df["venue"].unique():
    print(v)

M Chinnaswamy Stadium
Punjab Cricket Association Stadium, Mohali
Feroz Shah Kotla
Wankhede Stadium
Eden Gardens
Sawai Mansingh Stadium
Rajiv Gandhi International Stadium, Uppal
MA Chidambaram Stadium, Chepauk
Dr DY Patil Sports Academy
Newlands
St George's Park
Kingsmead
SuperSport Park
Buffalo Park
New Wanderers Stadium
De Beers Diamond Oval
OUTsurance Oval
Brabourne Stadium
Sardar Patel Stadium, Motera
Barabati Stadium
Vidarbha Cricket Association Stadium, Jamtha
Himachal Pradesh Cricket Association Stadium
Nehru Stadium
Holkar Cricket Stadium
Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium
Subrata Roy Sahara Stadium
Shaheed Veer Narayan Singh International Stadium
JSCA International Stadium Complex
Sheikh Zayed Stadium
Sharjah Cricket Stadium
Dubai International Cricket Stadium
Maharashtra Cricket Association Stadium
Punjab Cricket Association IS Bindra Stadium, Mohali
Saurashtra Cricket Association Stadium
Green Park
M.Chinnaswamy Stadium


In [22]:
df["venue"] = df["venue"].apply(lambda x:"M.Chinnaswamy Stadium" \
                                if x=="M Chinnaswamy Stadium" else x)

In [23]:
for v in df["venue"].unique():
    print(v)

M.Chinnaswamy Stadium
Punjab Cricket Association Stadium, Mohali
Feroz Shah Kotla
Wankhede Stadium
Eden Gardens
Sawai Mansingh Stadium
Rajiv Gandhi International Stadium, Uppal
MA Chidambaram Stadium, Chepauk
Dr DY Patil Sports Academy
Newlands
St George's Park
Kingsmead
SuperSport Park
Buffalo Park
New Wanderers Stadium
De Beers Diamond Oval
OUTsurance Oval
Brabourne Stadium
Sardar Patel Stadium, Motera
Barabati Stadium
Vidarbha Cricket Association Stadium, Jamtha
Himachal Pradesh Cricket Association Stadium
Nehru Stadium
Holkar Cricket Stadium
Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium
Subrata Roy Sahara Stadium
Shaheed Veer Narayan Singh International Stadium
JSCA International Stadium Complex
Sheikh Zayed Stadium
Sharjah Cricket Stadium
Dubai International Cricket Stadium
Maharashtra Cricket Association Stadium
Punjab Cricket Association IS Bindra Stadium, Mohali
Saurashtra Cricket Association Stadium
Green Park


In [24]:
df.columns

Index(['id', 'city', 'date', 'player_of_match', 'venue', 'team1', 'team2',
       'toss_winner', 'toss_decision', 'winner', 'result', 'result_margin',
       'umpire1', 'umpire2'],
      dtype='object')

## That's it :)

In [25]:
df.to_csv("matches.csv",index=False)